## 抓取MPC上小行星的数据
作者：AstroHHQ

### request 实现

load import & minorplanet list

In [24]:
#load import
import numpy as np
import pandas as pd
from urllib.request import urlopen
from requests.api import request

mpcdata = pd.read_table('mpc.txt')
#mpcdata.sort_values('Diam',ascending=False)
sort_data = mpcdata.sort_values('Diam',ascending=False)
#mpcdata['Name'][0]
mp_name = mpcdata.loc[:,['Name','Diam']]
mp_name.insert(2,'WiseData_time',0)
mp_name


,Name,Diam,WiseData_time
0,1991 VA,0.017,0
1,1994 CJ1,0.156,0
2,1995 CR,0.130,0
3,1995 HM,0.094,0
4,1996 GQ,0.071,0
...,...,...,...
1310,2011 LT17,0.134,0
1311,2014 AY28,0.122,0
1312,2014 QX266,0.091,0
1313,2015 BY310,0.130,0


creat a new list of minorplanet(record is there wise data and date)

In [34]:
import requests as rq
from tqdm import tqdm
#space to "+"
#name = mp_name[5]
#name = name.replace(' ','+')
lenMp = len(mp_name)
total = 0
for i in tqdm(range(lenMp)):
   name = mp_name['Name'][i]
   name = name.replace(' ','+')
   #get url from mpc
   url = f'https://minorplanetcenter.net/db_search/show_object?utf8=%E2%9C%93&object_id={name}'
   head ={
      'User-Agent':'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:94.0) Gecko/20100101 Firefox/94.0'
   }
   resp = rq.get(url,headers=head)
   #get html table
   mptable = pd.read_html(resp.text,match='Location')
   df = mptable[0]
   #is there wise data?
   dfyes = df[df['Location']=='C51 – WISE']
   if len(dfyes)>=1:
      dfout = dfyes["Date (UT)"]
      total = total + 1
      #get 1st date
      mp_name['WiseData_time'][i] = dfout[dfout.index[0]]
   else:
      mp_name['WiseData_time'][i] = -1
print(f'{total} wise data in {lenMp}') 
mp_name  


  1%|          | 16/1315 [00:29<40:24,  1.87s/it]


ValueError: No tables found matching pattern 'Location'

In [40]:
mp_name.loc[0:17]

,Name,Diam,WiseData_time
0,1991 VA,0.017,-1
1,1994 CJ1,0.156,-1
2,1995 CR,0.130,-1
3,1995 HM,0.094,-1
4,1996 GQ,0.071,2010 02 05.963786
5,1996 VB3,0.102,-1
6,1997 UR,0.068,-1
7,1998 BY7,0.151,-1
8,1998 KY26,0.024,-1
9,1998 WB2,0.113,-1


上面发现只有16号名字不符合 ，所以后面从第17个开始循环,total = 2

In [41]:
import requests as rq
from tqdm import tqdm
import time
#space to "+"
#name = mp_name[5]
#name = name.replace(' ','+')
lenMp = len(mp_name)
total = 2
for i in tqdm(range(17,lenMp)):
   if i%20==0:time.sleep(1)
   name = mp_name['Name'][i]
   name = name.replace(' ','+')
   #get url from mpc
   url = f'https://minorplanetcenter.net/db_search/show_object?utf8=%E2%9C%93&object_id={name}'
   head ={
      'User-Agent':'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:94.0) Gecko/20100101 Firefox/94.0'
   }
   resp = rq.get(url,headers=head)
   #get html table
   mptable = pd.read_html(resp.text,match='Location')
   df = mptable[0]
   #is there wise data?
   dfyes = df[df['Location']=='C51 – WISE']
   if len(dfyes)>=1:
      dfout = dfyes["Date (UT)"]
      total = total + 1
      #get 1st date
      mp_name['WiseData_time'][i] = dfout[dfout.index[0]]
   else:
      mp_name['WiseData_time'][i] = -1
print(f'{total} wise data in {lenMp}') 
mp_name  


  1%|          | 15/1298 [00:28<42:32,  1.99s/it]